Agent version example

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# Set environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo")
embedding = OpenAIEmbeddings()

In [3]:
from langchain_community.vectorstores import Chroma
import document_handler

# https://python.langchain.com/docs/integrations/vectorstores/chroma

chroma_collection_name = "LangChainCollection"

import chromadb
new_client = chromadb.EphemeralClient()

vectorstore_initialize = Chroma.from_documents(
    document_handler.processed_texts,
    embedding=embedding,
    collection_name=chroma_collection_name,
    client=new_client,
)

vectorstore = Chroma(
    client=new_client,
    collection_name=chroma_collection_name,
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

<class 'langchain_community.vectorstores.chroma.Chroma'>


In [4]:
docs = vectorstore_initialize.similarity_search("What is Chocolate?")
print(docs)
docs = vectorstore.similarity_search("What is Chocolate?")
print(docs)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, tool
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [29]:
from langchain.memory import ConversationSummaryBufferMemory
conversational_memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=model,
    memory_key = "chat_history",
    return_messages=True
)

In [41]:
# Prompt

system_message_template = (
    "You are a helpful assistant who helps answer questions. Answer only the facts based on the context. "
    "Your goal is to provide accurate and relevant answers based on the facts in the provided context. "
    "Make sure to reference the above source documents appropriately and avoid making assumptions or adding personal opinions. "
    "Emphasize the use of facts from the provided source documents. "
    "Instruct the model to use source name for each fact used in the response. "
    "Avoid generating speculative or generalized information. "
    "Use square brackets to reference the source, e.g. [info1.txt]. "
    "Do not combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"
    "Here is how you should answer every question:\n"
        "-Look for relevant information in the above source documents to answer the question.\n"
        "-If the source document does not include the exact answer, please respond with relevant information from the data in the response along with citation. You must include a citation to each document referenced.\n"
        "-If you cannot find answer in below sources, respond with I am not sure. Do not provide personal opinions or assumptions and do not include citations.\n"
        "-If you use any information in the context, include the index(starts at 1) of the statement as citation in your answer\n"
    "At the end of your response:\n" 
    "1. Add key words from the paragraphs. \n"
    "2. Suggest a further question that can be answered by the paragraphs provided. \n"
    "3. Create a source list of source name, author name, and a link for each document you cited.\n"
    "{context}"

)

MEMORY_KEY = "chat_history"

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_template),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("human", "{question}"),
        # MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [8]:
from langchain.tools.retriever import create_retriever_tool
# tools
retriever_tool = create_retriever_tool(
    retriever,
    "search_docs",
    "Searches and returns documents for information to answer the question",
)

@tool
def retrieve_documents_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)[:3]
    # return docs
    texts = [doc.page_content for doc in docs]
    texts_merged = '\n\n'.join(texts)
    # print("Retrieving documents")
    # print(texts_merged)
    return texts_merged


tools = [retriever_tool]

In [31]:
# Set llm chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# agent = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | final_prompt
#     | model
#     | StrOutputParser()
# )
from langchain.agents.conversational_chat.base import ConversationalChatAgent

agent = ConversationalChatAgent.from_llm_and_tools(llm=model, tools=tools, prompt=final_prompt)

In [43]:
agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=True, memory=conversational_memory,)

In [38]:
query = "What is chocolate?"
print(agent_executor.invoke(query))
conversational_memory.clear()



> Entering new AgentExecutor chain...
{
  "action": "search_docs",
  "action_input": "What is chocolate?"
}

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe

ValueError: One output key expected, got dict_keys(['output', 'intermediate_steps'])

In [12]:
# This doesn't work
from trulens_eval import TruChain, Feedback, Tru
from trulens_eval.schema import FeedbackResult
tru = Tru()
tru.reset_database()
from trulens_eval.feedback.provider import OpenAI
import numpy as np

# Initialize provider class
openai = OpenAI(model_engine="gpt-3.5-turbo")

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(agent_executor)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI(model_engine="gpt-3.5-turbo"))
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
    )

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


ValueError: Cannot find any `BaseRetriever` in app.

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI(model_engine="gpt-3.5-turbo")

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [15]:
# imports required for tracking and evaluation
from trulens_eval import Feedback, OpenAI, Tru, TruLlama, Select, OpenAI as fOpenAI
from trulens_eval.feedback import GroundTruthAgreement, Groundedness

tru = Tru()

In [20]:
import openai
client = openai.OpenAI()

chat_completion = client.chat.completions.create

# from trulens_eval.tru_custom_app import TruCustomApp, instrument

# class LLMStandaloneApp():
#     @instrument
#     def __call__(self, prompt):
#         return chat_completion(
#             model="gpt-3.5-turbo",
#             messages=[
#                     {"role": "system", "content": gordon_ramsay_prompt},
#                     {"role": "user", "content": prompt}
#                 ]
#         ).choices[0].message.content

# llm_standalone = LLMStandaloneApp()

In [21]:
class OpenAI_custom(OpenAI):
    def query_translation_score(self, question1: str, question2: str) -> float:
        return float(chat_completion(
            model="gpt-3.5-turbo",
            messages=[
                    {"role": "system", "content": "Your job is to rate how similar two quesitons are on a scale of 1 to 10. Respond with the number only."},
                    {"role": "user", "content": f"QUESTION 1: {question1}; QUESTION 2: {question2}"}
                ]
        ).choices[0].message.content) / 10

    def ratings_usage(self, last_context: str) -> float:
        return float(chat_completion(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Your job is to respond with a '1' if the following statement mentions ratings or reviews, and a '0' if not."},
                {"role": "user", "content": f"STATEMENT: {last_context}"}
            ]
        ).choices[0].message.content)

In [33]:
custom = OpenAI_custom()
# Input to tool based on trimmed user input.
f_query_translation = Feedback(
    custom.query_translation_score,
    name="Query Translation") \
.on_input() \
.on(Select.RecordCalls.agent.plan.args.kwargs.input)

# f_ratings_usage = Feedback(
#     custom.ratings_usage,
#     name="Ratings Usage") \
# .on(Select.Record.app.query[0].rets.response)

# Result of this prompt: Given the context information and not prior knowledge, answer the query.
# Query: address of Gumbo Social
# Answer: "
fopenai = fOpenAI()
# Question/statement (context) relevance between question and last context chunk (i.e. summary)
f_context_relevance = Feedback(
    fopenai.qs_relevance,
    name="Context Relevance") \
.on_input() \
.on(Select.RecordCalls.tools[0]._run.rets)

# Groundedness
grounded = Groundedness(groundedness_provider=fopenai)

f_groundedness = Feedback(
    grounded.groundedness_measure,
    name="Groundedness") \
.on(Select.RecordCalls.tools[0]._run.rets) \
.on_output().aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    fopenai.relevance,
    name="Answer Relevance"
).on_input_output()

✅ In Query Translation, input question1 will be set to __record__.main_input or `Select.RecordInput` .
✅ In Query Translation, input question2 will be set to __record__.app.agent.plan.args.kwargs.input .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.tools[0]._run.rets .
✅ In Groundedness, input source will be set to __record__.app.tools[0]._run.rets .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [34]:
tru_recorder = TruChain(agent_executor,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_query_translation, f_qa_relevance, f_context_relevance, f_groundedness])

In [44]:
with tru_recorder as recording:
    try:
        llm_response = agent_executor.invoke({"What is Chocolate?"})
        print(llm_response)
    except:
        print(recording)

conversational_memory.clear()

A new object of type <class 'langchain.agents.agent.AgentExecutor'> at 0x18081358910 is calling an instrumented method <function Chain.invoke at 0x00000180E72E1300>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x180ff748640) using this function.
A new object of type <class 'langchain.agents.agent.AgentExecutor'> at 0x18081358910 is calling an instrumented method <function AgentExecutor._call at 0x00000180E7CA1A80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x180ff748640) using this function.
A new object of type <class 'langchain.agents.conversational_chat.base.ConversationalChatAgent'> at 0x1808118b650 is calling an instrumented method <function Agent.plan at 0x00000180E7C6BF60>. The path of this call may be incorrect.
Guessing path of new object is app.agent based on other object (0x180fface250) using this function.




> Entering new AgentExecutor chain...


A new object of type <class 'langchain_core.tools.Tool'> at 0x1808135a7b0 is calling an instrumented method <function Tool._run at 0x00000180D0E9AFC0>. The path of this call may be incorrect.
Guessing path of new object is app.tools[0] based on other object (0x180ffb0abc0) using this function.


{
    "action": "search_docs",
    "action_input": "What is Chocolate?"
}

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


...


Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe

A new object of type <class 'langchain.memory.summary_buffer.ConversationSummaryBufferMemory'> at 0x18081367550 is calling an instrumented method <function ConversationSummaryBufferMemory.save_context at 0x00000180E78D2FC0>. The path of this call may be incorrect.
Guessing path of new object is app.memory based on other object (0x180fef85b90) using this function.
A new object of type <class 'langchain.memory.summary_buffer.ConversationSummaryBufferMemory'> at 0x18081367550 is calling an instrumented method <function BaseChatMemory.save_context at 0x00000180E785D760>. The path of this call may be incorrect.
Guessing path of new object is app.memory based on other object (0x180fef85b90) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x18082686c10 is calling an instrumented method <function Chain.__call__ at 0x00000180E72E1BC0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x180ff748640) using this 

{
    "action": "Final Answer",
    "action_input": "Chocolate is a sweet, usually brown, food product made from cocoa beans. It is used in confectionery and desserts and comes in various forms such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate has a higher cocoa content and less sugar, while milk chocolate includes milk solids for a sweeter taste. White chocolate mainly consists of cocoa butter, sugar, and milk solids."
}

> Finished chain.


A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x18082686c10 is calling an instrumented method <function Chain.invoke at 0x00000180E72E1300>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x180ff748640) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x18082686c10 is calling an instrumented method <function LLMChain._call at 0x00000180E73267A0>. The path of this call may be incorrect.
Guessing path of new object is app.agent.llm_chain based on other object (0x180ffacbc00) using this function.
Unsure what the main input string is for the call to invoke with args [{'What is Chocolate?'}].


{'input': {'What is Chocolate?'}, 'chat_history': [], 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is used in confectionery and desserts and comes in various forms such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate has a higher cocoa content and less sugar, while milk chocolate includes milk solids for a sweeter taste. White chocolate mainly consists of cocoa butter, sugar, and milk solids.'}


In [40]:
display(llm_response)
# The record of the app invocation can be retrieved from the `recording`:

{'input': {'What is Chocolate?'},
 'chat_history': [SystemMessage(content='The human asks what chocolate is. The AI explains that chocolate is a sweet, usually brown, food product made from cocoa beans. It comes in various forms, including dark chocolate, milk chocolate, and white chocolate. Dark chocolate has a higher cocoa content, while milk chocolate is sweeter due to the addition of milk solids. White chocolate mainly consists of cocoa butter, sugar, and milk solids but does not contain cocoa solids.')],
 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It comes in various forms, including dark chocolate, milk chocolate, and white chocolate. Dark chocolate has a higher cocoa content, while milk chocolate is sweeter due to the addition of milk solids. White chocolate mainly consists of cocoa butter, sugar, and milk solids but does not contain cocoa solids.'}

In [21]:
rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

Record(record_id='record_hash_c948c70c94c7025ad326b0a9c3baa3e9', app_id='Chain1_ChatApplication', cost=Cost(n_requests=4, n_successful_requests=4, n_classes=0, n_tokens=1535, n_stream_chunks=0, n_prompt_tokens=1444, n_completion_tokens=91, cost=0.0023419999999999995), perf=Perf(start_time=datetime.datetime(2024, 1, 31, 16, 40, 1, 105521), end_time=datetime.datetime(2024, 1, 31, 16, 40, 9, 897672)), ts=datetime.datetime(2024, 1, 31, 16, 40, 9, 910204), tags='-', meta=None, main_input=['What is Chocolate?'], main_output='Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate.', main_error=None, calls=[RecordAppCall(stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain.agents.agent.AgentExecutor, id=1939389171584, init_bindings=None), name='invoke')), RecordAppCallMethod(pa

In [22]:
# The results of the feedback functions can be rertireved from the record. These
# are `Future` instances (see `concurrent.futures`). You can use `as_completed`
# to wait until they have finished evaluating.

from concurrent.futures import as_completed

for feedback_future in  as_completed(rec.feedback_results):
    feedback, feedback_result = feedback_future.result()
    
    feedback: Feedback
    feedbac_result: FeedbackResult

    display(feedback.name, feedback_result.result)

    print(feedback_result)

'relevance'

1.0

feedback_result_id='feedback_result_hash_a7dec5f491adabb462010b65aff30f46' record_id='record_hash_c948c70c94c7025ad326b0a9c3baa3e9' feedback_definition_id='feedback_definition_hash_6f9c9b1fb1c0bb27380a823df8a48b57' last_ts=datetime.datetime(2024, 1, 31, 16, 40, 10, 373948) status=<FeedbackResultStatus.DONE: 'done'> cost=Cost(n_requests=1, n_successful_requests=1, n_classes=0, n_tokens=392, n_stream_chunks=0, n_prompt_tokens=391, n_completion_tokens=1, cost=0.0005885) name='Answer Relevance' calls=[FeedbackCall(args={'prompt': ['What is Chocolate?'], 'response': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate.'}, ret=1.0, meta={})] result=1.0 error=None multi_result='null'


'query_translation_score'

0.8

feedback_result_id='feedback_result_hash_2a130ff9f54a80b31acb3ac1246a0f8b' record_id='record_hash_c948c70c94c7025ad326b0a9c3baa3e9' feedback_definition_id='feedback_definition_hash_92ab5787fd741929eab89e4acb9e74be' last_ts=datetime.datetime(2024, 1, 31, 16, 40, 10, 146810) status=<FeedbackResultStatus.DONE: 'done'> cost=Cost(n_requests=1, n_successful_requests=1, n_classes=0, n_tokens=108, n_stream_chunks=0, n_prompt_tokens=107, n_completion_tokens=1, cost=0.0001625) name='Query Translation' calls=[FeedbackCall(args={'question1': ['What is Chocolate?'], 'question2': {'tool': 'retrieve_documents_tool', 'tool_input': 'What is Chocolate?', 'log': '{\n    "action": "retrieve_documents_tool",\n    "action_input": "What is Chocolate?"\n}', 'type': 'AgentAction'}}, ret=0.8, meta={})] result=0.8 error=None multi_result='null'


In [19]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])
records.head(10)

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,Answer Relevance,Context Relevance,Groundedness,Query Translation_calls,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",AgentExecutor(langchain.agents.agent),record_hash_05258b9a836eca4ec61ac6786a05180c,"[""What is Chocolate?""]","""Chocolate is a sweet, usually brown, food pro...",-,"{""record_id"": ""record_hash_05258b9a836eca4ec61...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-01-31T17:25:30.236360"", ""...",...,1.0,1.0,1.0,[{'args': {'question1': ['What is Chocolate?']...,"[{'args': {'prompt': ['What is Chocolate?'], '...","[{'args': {'question': ['What is Chocolate?'],...","[{'args': {'source': 'Chocolate is a sweet, us...",6,1381,0.002105


In [24]:
tru.get_leaderboard(app_ids=["Chain1_ChatApplication"])

,Answer Relevance,Query Translation,latency,total_cost
app_id,,,,
Chain1_ChatApplication,1.0,0.8,8.0,0.002342


In [27]:
tru.run_dashboard()
# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.220:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Exception in thread Thread-16 (_future_target_wrapper):
Traceback (most recent call last):
  File "C:\Users\jiheu\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\jiheu\LLMTesting\LLMTesting\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jiheu\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\jiheu\LLMTesting\LLMTesting\.venv\Lib\site-packages\trulens_eval\utils\python.py", line 248, in _future_target_wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jiheu\LLMTesting\LLMTesting\.venv\Lib\site-packages\trulens_eval\tru.py", line 662, in listen_to_dashboard
    line = pipe.readline()
           ^^^^^^^^^^^^^^^
UnicodeDecodeError: 'cp949' codec can't decode byte 0xf0 in position 0: illegal multibyte sequence
